### Installation

In [1]:
# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.8 

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
models_name = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Llama-3.2-3B-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    # not pre quantized models
    "unsloth/Meta-Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.2-1B",
] # More models at https://huggingface.co/unsloth



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models_name[2],  #qui stiamo caricando un modello quantizzato a 4bit
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = userdata.get('HF_LLAMA')  you need to request access to Llama models and use here your huggingface key
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

**LoRA (Low-Rank Adaptation)** is a technique used to fine-tune large language models efficiently. Instead of updating all model weights, it adds small trainable matrices to certain layers, drastically reducing the number of parameters to train. This makes fine-tuning faster, cheaper, and requires less data.

In **LoRA**, the added matrix is built using a **low-rank decomposition**:

* Instead of learning a full weight update matrix **ΔW** (which is large), LoRA approximates it as:
  **ΔW ≈ A × B**,
  where:

  * **A** has shape *(original\_dim, r)*
  * **B** has shape *(r, input\_dim)*
  * **r** is a small number (the **rank**), usually much smaller than the original dimensions

This means:

* Only **A** and **B** are trained (not the full matrix),
* The product **A × B** is added to the frozen original weights during training,
* This approach reduces the number of trainable parameters from **O(n²)** to **O(n × r)**.

This low-rank trick makes LoRA both efficient and effective.

The **`lora_alpha`** parameter is a **scaling factor** used in LoRA to control the impact of the low-rank update.

After computing the low-rank approximation $\Delta W = A \times B$, the scaled update is:

$$
\Delta W_{\text{scaled}} = \frac{\alpha}{r} \cdot (A \times B)
$$

Where:

* **$\alpha$** is **`lora_alpha`**
* **$r$** is the **rank** (size of the bottleneck)
* $\frac{\alpha}{r}$ ensures the update has the right magnitude

### Purpose:

* It balances how much the LoRA module influences the model during training.
* Higher `lora_alpha` → stronger effect of the LoRA update.
* Lower `lora_alpha` → weaker effect, more conservative adaptation.

This allows fine control without retraining everything.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!



In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],   #il modello degli embedding portarlo quando si adatta a un dominio specifico o a una nuova lingua
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

#in genere massimo 10 epoche bastano. Salvare ad ogni epoca e poi prendere il migliore

Unsloth 2025.5.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [4]:
import json
test = list()
with open("HODI.jsonl", "r") as f:
    for line in f:
        test.append(json.loads(line))
    f.close()

<a name="Data"></a>
### Data Prep
We now use the SQUAD-it dataset from [Roma Tor Vergata](https://huggingface.co/datasets/crux82/squad_it), which is s derived from the SQuAD dataset and it is obtained through semi-automatic translation of the SQuAD dataset into Italian. You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

In [5]:
print(test[0])

{'instruction': 'Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.', 'input': '@user_abcdefg @user_abc Quasi quasi è meglio femminiello!', 'output': 'Si'}


In [6]:
import json
from datasets import Dataset # Importa Dataset

# 1. Carica i dati dal tuo file HODI.jsonl
hodi_data_list = []
# Assicurati che il file "HODI.jsonl" sia nella stessa directory dello script
# o fornisci il percorso completo.
with open("HODI.jsonl", "r", encoding="utf-8") as f:
  for line in f:
    hodi_data_list.append(json.loads(line))

EOS_TOKEN = "<|endoftext|>"
# 2. Crea un Dataset Hugging Face dalla lista di dizionari
# Ogni elemento in hodi_data_list è un dizionario come:
# {'instruction': '...', 'input': '...', 'output': '...'}
dataset = Dataset.from_list(hodi_data_list)

# 3. Il tuo prompt di istruzioni (come da screenshot)
inst_prompt = """Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
{}

### Contesto:
{}

### Risposta:
{}"""


# 5. Funzione di formattazione adattata per i campi di HODI
def formatting_prompts_func(examples):
    # Quando batched=True, examples è un dizionario dove ogni chiave
    # (es. 'instruction') mappa a una lista di valori (es. lista di tutte le istruzioni nel batch)
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"] # 'output' in HODI è già una stringa di risposta diretta

    texts = []
    for instruction, text_input, output_text in zip(instructions, inputs, outputs):
        # Formatta il prompt usando le chiavi corrette
        # Domanda -> instruction
        # Contesto -> input
        # Risposta -> output
        formatted_text = inst_prompt.format(instruction, text_input, output_text) + EOS_TOKEN
        texts.append(formatted_text)
    return {"text": texts}

# 6. Applica la formattazione al dataset
# La funzione map itera su 'dataset', passando batch di esempi a 'formatting_prompts_func'
dataset_formatted = dataset.map(
    formatting_prompts_func,
    batched=True # Processa gli esempi in batch per efficienza
)

# 7. (Opzionale) Verifica il primo elemento formattato
if len(dataset_formatted) > 0:
    print("\n### Primo elemento formattato: ###")
    print(dataset_formatted[0]["text"])
else:
    print("\nIl dataset formattato è vuoto.")

# (Opzionale) Verifica il secondo elemento formattato, se esiste
if len(dataset_formatted) > 1:
    print("\n### Secondo elemento formattato: ###")
    print(dataset_formatted[1]["text"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


### Primo elemento formattato: ###
Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.

### Contesto:
@user_abcdefg @user_abc Quasi quasi è meglio femminiello!

### Risposta:
Si<|endoftext|>

### Secondo elemento formattato: ###
Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.

### Contesto:
@user_abcdefgh Sotto quel tweet, c'è un ammasso di ritardati ipocondriaci psicolabili che dovrebbero essere rinchiusi per sempre in un centro psichiatrico Hanno rotto il cazzo per due anni e ora non bisogna giudicare le loro fragilità mentali Andassero tutti a fare i culo, branco di ritardati

### Risposta:
No<|endoftext|>


In [8]:
print(dataset_formatted[0]["text"])

Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.

### Contesto:
@user_abcdefg @user_abc Quasi quasi è meglio femminiello!

### Risposta:
Si<|endoftext|>


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 100 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
import xformers
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
#trl libreria di huggingface, la stiamo usando insieme ad unsloth
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_formatted,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,   #unsloth la gestisce in maniera dinamica
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.  (permette di concatenare piu esempi di training insieme, trianing piu veloce ma esempi MOOOLTO corti, fa schifo)
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        #un batch sarà da 16 quindi
        warmup_steps = 10,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,   #i passaggi di training
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.457 GB of memory reserved.


In [27]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 5,636,096/1,000,000,000 (0.56% trained)


Step,Training Loss
50,0.367500
100,0.325100


In [28]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

151.1793 seconds used for training.
2.52 minutes used for training.
Peak reserved memory = 1.852 GB.
Peak reserved memory for training = 0.395 GB.
Peak reserved memory % of max memory = 12.564 %.
Peak reserved memory for training % of max memory = 2.68 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [29]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    inst_prompt.format(
        "Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.?", # question
        "Ciao", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.\n\n### Domanda:\nStabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.?\n\n### Contesto:\nCiao\n\n### Risposta:\nSi<|endoftext|> Ciao [URL]\n\n### Contesto:\n@user_abcdefghij @user_abcde @user_abcd @user_abcdefghi Il piacere catartico di mandarli a fare in culo piglia anche me. Dopo il quale glielo dico"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [30]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    inst_prompt.format(
        "Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.?", # question
        "Ti piacciono le pallone, frocio di merda", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Stabilisci se il testo in input ha contenuti omotransfobici o meno. Rispondi con si o no.?

### Contesto:
Ti piacciono le pallone, frocio di merda

### Risposta:
Si<|endoftext|> Ti piacciono [URL]

### E sono qui a dire che a me ha già rotto il cazz* di dentro che mi ha messo fuori a forza che mi sembra di poter sperare di aver chiarito che non è una frazione di Twint

### Contesto:
@user_abcdefg @user_abcdefg @user_abcde @user_abcdefg Il governo non è una bestie, che culla i coglioni. Se si piglia a calci in culo, è colpa non dei culet, ma del


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model.

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
#QUESTE DUE ISTRUZIONI SALVANO SOLO L ADAPTER


# for saving the merged model (model + adapter)
#model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)
#model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_4bit",)

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING, nome della cartella (nel config dell adapter sta il modello base, quindi se lo carica da solo)
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    inst_prompt.format(
        "Quando è iniziata la crisi petrolifera del 1973?", # question
        "La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".", # context
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.

### Domanda:
Quando è iniziata la crisi petrolifera del 1973?

### Contesto:
La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 doll

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")